In [ ]:
# Setup code for Google Colab
USE_COLAB = True
GOOGLE_DRIVE_PATH = ''
if USE_COLAB:
    print("Using Colab!")
    %load_ext autoreload
    %autoreload 2

    from google.colab import drive
    drive.mount('/content/drive')

    import os
    GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Fall20/ML Project'
    GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
    print(os.listdir(GOOGLE_DRIVE_PATH))

    import sys
    sys.path.append(GOOGLE_DRIVE_PATH)

    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    import torchvision
    import statistics
    import random
    import time
    import math
    import numpy as np
    import cv2
    import copy
    import shutil
    import os
    import json

    import matplotlib.pyplot as plt
    %matplotlib inline

    !pip3 install -q idx2numpy

    if torch.cuda.is_available:
      print('Good to go!')
    else:
      print('Please set GPU via Edit -> Notebook Settings.')

In [ ]:
import Mnist_loader as mnist

x_train, y_train, x_test, y_test = mnist.load_odd_even_Mnist(USE_COLAB=True, path=GOOGLE_DRIVE_PATH)

In [ ]:

Ks = [1,2,4,8,10]
lambs = [1,5,10,20,50,100]
lrs = [1e-3, 1e-2, 5e-2, 1e-1, 5e-1, 1]

folds = 10

N = x_train.shape[0]

indices = torch.randperm(N)
length = N // folds

max_acc = 0.0
max_reg = 0.0
max_lr = 0.0

In [1]:
from CSVM_class import *
from Evaluate_models import *

for K in Ks:
    for lamb in lambs:
        for lr in lrs:
            acc = torch.zeros(folds)
            for f in range(folds):
                x_cross = torch.cat((x_train[indicies[0:f*length]],x_train[indicies[(f+1)*length:]]),dim=0)
                y_cross = torch.cat((y_train[indicies[0:f*length]],y_train[indicies[(f+1)*length:]]),dim=0)
                x_val = x[indicies[f*length:(f+1)*length]]
                y_val = y[indicies[f*length:(f+1)*length]]

                model = ClusterSVM(K=K, lamb=lamb)
                model.train(x_cross,y_cross, learning_rate=lr)

                y_pred = model.predict(x_val)
                acc[f] = ((y_val==y_pred).sum()) / float(y_val.shape[0])
            
            acc_mean = acc.mean()
            if(acc_mena > max_acc):
                max_acc = acc_mean
                max_lamb = lamb
                max_lr = lr
                max_K = K

            print("at K = {}, lamb = {}, lr = {}: get acc = {}".format(K,lamb,lr,acc_mean))

print("Maximum accuracy = {} at: K = {}, lamb = {}, lr = {}".format(max_acc,max_K,max_l))


ModuleNotFoundError: No module named 'Evaluate_model'

In [ ]:
model = ClusterSVM(K=max_K,lamb=max_lamb)

comp_time, acc = evaluate_model(model,x_train,y_train,x_test,ytest)